In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [ ]:
class Time2Vec(nn.Module):
  def __init__(self, num_frequency=2, period=1.0):
    super(Time2Vec,self).__init__()
    self.num_frequency = num_frequency
    self.period = period
    # self.freqs = torch.linspace(1, num_frequency,num_frequency).float()
    self.freqs=torch.tensor([1,5])

  def forward(self,t):
    t = t.reshape(-1, 1)
    sin_features = torch.sin(2 * torch.pi * self.freqs.reshape(1, -1) * t / self.period)
    cos_features = torch.cos(2 * torch.pi * self.freqs.reshape(1, -1) * t / self.period)
    time_features = torch.cat([sin_features, cos_features, t], dim=1)
    return time_features



In [ ]:
class StockTransformer(nn.Module):
  def __init__(self, input_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, output_dim):
    super(StockTransformer, self).__init__()
    self.transformer_encoder = nn.TransformerEncoder(
        nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead),
        num_layers=num_encoder_layers
    )
    self.transformer_decoder = nn.TransformerDecoder(
          nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead),
          num_layers=num_decoder_layers
    )
    self.fc_out=nn.Linear(d_model,output_dim)

    def forward(self):
        src = src.permute(1, 0, 2)
        tgt = tgt.permute(1, 0, 2)
        memory = self.transformer_encoder(src)
        output = self.transformer_decoder(tgt, memory)



In [ ]:
def create_sequences(input,in_seq_length=100,out_seq_len=5):
  x,y=[],[]
  for i in range(len(input)-in_seq_length-out_seq_len):
    x.append(input[i:i+in_seq_length])
  for i in range(len(input)-in_seq_length-out_seq_len):
    y.append(input[i+in_seq_length:i+in_seq_length+out_seq_len])
  return np.array(x),np.array(y)



In [ ]:
tata_mot=yf.Ticker('TATAMOTORS.BO')

In [ ]:
tata_motors=tata_mot.history(period='max')

In [ ]:
tata_motors=yf.download('TATAMOTORS.BO',period='max')

[*********************100%***********************]  1 of 1 completed


In [ ]:
tata_motors.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,43.5,43.5,43.5,43.5,16.015549,0
2000-01-04,43.5,43.5,43.5,43.5,16.015549,0
2000-01-05,43.5,43.5,43.5,43.5,16.015549,0
2000-01-06,43.5,43.5,43.5,43.5,16.015549,0
2000-01-07,43.5,43.5,43.5,43.5,16.015549,0


In [ ]:
# tata_motors['12day_EMA']=tata_motors['Close'].ewm(span=12, adjust=False).mean()
# tata_motors['24day_EMA']=tata_motors['Close'].ewm(span=24,adjust=False).mean()
# tata_motors['MACD']=tata_motors['12day_EMA']-tata_motors['24day_EMA']

In [ ]:
# delta = tata_motors['Close'].diff()
# gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
# loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
# rs = gain / loss
# tata_motors['RSI'] = 100 - (100 / (1 + rs))

In [ ]:
timestamps=tata_motors.index

In [ ]:
type(timestamps)

pandas.core.indexes.datetimes.DatetimeIndex

In [ ]:
timestamps=pd.Series(timestamps)

In [ ]:
timestamps=timestamps.apply(lambda x : x.toordinal())

In [ ]:
date_tensor=torch.tensor(timestamps).float()
features_tensor=torch.tensor(tata_motors.to_numpy()).float()

In [ ]:
time2vec_layer=Time2Vec(num_frequency=2)

In [ ]:
time_features=time2vec_layer(date_tensor)

In [ ]:
time_features.shape

torch.Size([6129, 5])

In [ ]:
input=torch.cat([time_features,features_tensor],dim=1)

In [ ]:
x,y=create_sequences(input)

In [ ]:
x.shape

(6024, 100, 11)

In [ ]:
tata_motors.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,43.5,43.5,43.5,43.5,16.015549,0
2000-01-04,43.5,43.5,43.5,43.5,16.015549,0
2000-01-05,43.5,43.5,43.5,43.5,16.015549,0
2000-01-06,43.5,43.5,43.5,43.5,16.015549,0
2000-01-07,43.5,43.5,43.5,43.5,16.015549,0
